# Домашнее задание №2: линейная регрессия (10 баллов).

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [1]:
import numpy as np

### Считаю свой вариант

In [2]:
print((len("Kazakbaev") % 4) + 1)

2


## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [3]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
mse_lin = mean_squared_error(y, reg.predict(X))
print(mse_lin)

1.124696681990535e-25


In [5]:
np.mean((y-reg.predict(X))**2)

1.124696681990535e-25

Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [15]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

2.338983326802504e-12


array([-9.88707175e-09,  2.23074159e-08,  5.34775968e-08, -2.55072391e-09,
        6.82681003e-09, -1.32802674e-08, -1.32966179e-08, -8.57583660e-09,
        4.61120859e-08, -2.32182314e-08,  3.70500401e+01, -2.64778235e-08,
       -1.22552274e-08,  7.48625116e-09,  4.36060209e-08, -3.97312495e-08,
       -7.62927819e-08, -4.01389487e-08,  1.39682803e+01,  2.30230716e+01,
        1.87018531e-08, -1.17501384e-08, -1.02049391e-08,  2.11091429e-08,
       -2.22806107e-09,  1.61346019e-08,  6.31415153e-09,  3.52243407e-09,
        7.73774363e-09,  2.36654128e-08, -3.82708042e-08, -6.89005672e-09,
       -1.26548462e-08, -4.39147308e-08, -1.18917891e-08, -3.09392044e-08,
        2.89222127e-08,  1.43208119e-08,  2.42285928e-08, -1.03792384e-08,
        2.59962993e+01,  1.25040507e-08,  4.64551127e+01, -2.87109812e-08,
        9.70673003e+01,  3.02971232e-08, -6.03194484e-09,  4.96982251e-08,
        3.10850210e-08, -3.40031774e-08, -3.76208805e-08,  3.58790792e-08,
       -5.60861233e-08,  

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

### Задание 1

Обратимся, к документации. Просто Linear Regression минимизирует квадрат разницы того, что неблюдается и то, что предсказано. В то же время, как SGDRegressor сдвигается в сторону антиградиента с определенным шагом - learning rate, и с наличием penalty. У SGDRegressor есть penalty который по дефолту стоит L2, у Linear regression этого нет. 

Коэффицент $\alpha$ это множитель регулирязатора(penalty), то есть то насколько сильно мы штрафуем за слишком большие веса. Уменьшая/увеличивая $\alpha$ будем получать разные MSE.

Соответсвенно будет и другой минимизируемая функция: $Q = \frac {1} {l} \sum ((w, x_i) - y_i) ^ 2 + \alpha * \sum (w_i) ^2$

И соотвественно разные MSE

### Задание 2

In [19]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

2.2764718923605236e-12


Это наиболее близкое, что я подбором смог сделать. Сокращая альфу, мы сокращаем MSE. В любом случае получившиеся значение это очень маленькое число.

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию. 

### Задание 3

In [9]:
import sys
epsilon = sys.float_info.epsilon
class LinearRegression(object):
    def __init__(self, learning_rate=0.01, l_ratio=0.001, tol=0.001, epochs=1500, weights=np.zeros(X.shape[1]),
        costs = [0]):
        '''
        Для начала необходимо инициализировать параметры
        learning_rate - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.learning_rate = learning_rate
        self.l_ratio = l_ratio
        self.tol = tol
        self.epochs = epochs  
        self.costs = costs
        self.weights = weights
    def fit(self, X, y_true = y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        weights = self.weights
        costs = self.costs
        learning_rate = self.learning_rate
        N = float(len(y_true))
        for i in range(self.epochs):
            y_pred = np.dot(X, weights)
            cost = 1 / N * np.sum((y_true - y_pred)** 2)
            weights_gradient = (-2 / N) * np.dot(X.T, (y_true - y_pred)) 
            weights = weights - learning_rate * weights_gradient
            costs.append(cost)
            if abs(costs[-1] - costs[-2]) < epsilon:
                break
        return weights
   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        y_pr = np.dot(X, self.fit(X, y))
        return y_pr

In [10]:
my_reg = LinearRegression()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

You are amazing! Great work!


### Задание 4

In [11]:
import sys
epsilon = sys.float_info.epsilon
class LinearRegression_l2(object):
    def __init__(self, learning_rate=0.01, l_ratio=0.1, tol=0.001, epochs=10000, weights=np.zeros(X.shape[1]),
        costs = [0]):
        '''
        Для начала необходимо инициализировать параметры
        learning_rate - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.learning_rate = learning_rate
        self.l_ratio = l_ratio
        self.tol = tol
        self.epochs = epochs  
        self.costs = costs
        self.weights = weights
    def fit(self, X, y_true = y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        weights = self.weights
        costs = self.costs
        learning_rate = self.learning_rate
        l_ratio = self.l_ratio
        N = float(len(y_true))
        for i in range(self.epochs):
            y_pred = np.dot(X, weights)
            cost = 1 / N * np.sum((y_true - y_pred)** 2) + l_ratio * np.sum(weights ** 2)
            weights_gradient = (-2 / N) * np.dot(X.T, (y_true - y_pred)) + (l_ratio * 2 * weights) / N
            weights = weights - learning_rate * weights_gradient
            costs.append(cost)
            if abs(costs[-1] - costs[-2]) < epsilon:
                break
        return weights
   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        y_pr = np.dot(X, self.fit(X, y))
        return y_pr

In [12]:
my_reg = LinearRegression_l2()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work! even with l2 ratio')

You are amazing! Great work! even with l2 ratio


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и четвертый варианты) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй и третий варианты) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [13]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1).fit(X, y)
print('MSE:', mean_squared_error(y, ridge.predict(X)))
print('Weights:', ridge.coef_)

MSE: 2.304540549080835e-06
Weights: [ 1.44912814e-06 -1.40678817e-05  9.70612687e-06  5.58687665e-06
 -1.14165171e-05  5.92121595e-06 -5.68121728e-06 -2.25606353e-05
  7.64011158e-06 -4.40383365e-06  3.70496800e+01 -9.32097248e-06
 -5.45192966e-06 -3.86092962e-06  3.35516111e-06 -1.73841099e-05
 -4.87845232e-06 -1.44614508e-05  1.39681483e+01  2.30228151e+01
  2.13578079e-05 -1.29382940e-05  3.07403323e-05 -8.94947936e-06
  4.12011243e-06  1.13374918e-05 -3.27813478e-06 -4.83473953e-06
  1.63519856e-05  9.99138198e-07  8.97551827e-06 -1.85152318e-05
 -3.13605184e-05 -1.51604486e-05 -1.68707470e-05 -7.00610174e-06
  2.33408795e-06  9.73752982e-06  7.17135576e-06 -8.46942820e-06
  2.59960079e+01 -6.96844706e-06  4.64546467e+01 -2.78731343e-05
  9.70663623e+01 -4.06741221e-06 -9.49344741e-06 -8.06269277e-07
  6.68509241e-06 -4.19933593e-05 -3.15957390e-05  3.15899221e-06
 -1.75395754e-05  2.62305096e+01 -6.68419809e-06 -1.69394344e-06
  1.38114745e-05  3.67280909e-06 -1.24008620e-05  6.04

In [14]:
my_reg = LinearRegression_l2()
print('MSE:', mean_squared_error(y, my_reg.predict(X)))
print('Weights:', my_reg.fit(X, y))

MSE: 2.3045106007388436e-06
Weights: [ 1.44391714e-06 -1.40811132e-05  9.69690207e-06  5.63175961e-06
 -1.14262941e-05  6.00734738e-06 -5.64246212e-06 -2.26048990e-05
  7.70920670e-06 -4.37313713e-06  3.70496800e+01 -9.35245353e-06
 -5.40895020e-06 -3.82505803e-06  3.34849577e-06 -1.74279227e-05
 -4.83496481e-06 -1.45468800e-05  1.39681484e+01  2.30228151e+01
  2.13324623e-05 -1.29961190e-05  3.07695858e-05 -8.82124015e-06
  4.06549066e-06  1.13039332e-05 -3.28987091e-06 -4.74460016e-06
  1.63346379e-05  9.88384161e-07  9.05636925e-06 -1.84559008e-05
 -3.12814868e-05 -1.52415704e-05 -1.69805540e-05 -7.03282980e-06
  2.24349777e-06  9.68238861e-06  7.12345143e-06 -8.56979132e-06
  2.59960079e+01 -6.94842869e-06  4.64546468e+01 -2.78993954e-05
  9.70663623e+01 -4.06292261e-06 -9.43637996e-06 -6.74156627e-07
  6.75907137e-06 -4.18911525e-05 -3.16243252e-05  3.26055514e-06
 -1.74827320e-05  2.62305096e+01 -6.66441651e-06 -1.71439132e-06
  1.38025562e-05  3.69488841e-06 -1.23023368e-05  6.4

## Вывод

Получились почти что одинаковые MSE, значит все ок!